In [44]:
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
from hyperparameters import hyperparameters
import matplotlib as mpl

from IPython.display import display
plt.style.use('bmh')
mpl.rcParams.update({
    "grid.linestyle" : "dashed",
    "axes.facecolor" : "white",
    "axes.spines.top" : False,
    "axes.spines.right" : False,
    "legend.frameon" : False,
    "figure.figsize" : (8, 5),
    "figure.dpi" : 300,
})
%matplotlib inline

# Suppress sklearn deprecated warnings
import warnings
def warn(*args, **kwargs): pass
warnings.warn = warn
np.set_printoptions(threshold=sys.maxsize)

np.random.seed(42)

In [45]:
# TODO make test for dataset with creatina column
# Dataset without thyroid = 18 features (including survive7y)
# Dataset with thyroid = 27 features (including survive7y)
# With columns that have missing values, 23 and 32
# Default 18
n_features = 32
extra_path = n_features != 27 and n_features != 18
dropped_na_key = "dropped_na/"
mean_key = "mean/"
key = mean_key
path = f"data/{n_features}features/{key if extra_path else '' }"
path_models = f"models/{n_features}features/{key if extra_path else '' }"
output_models = f"models_output/{n_features}features/{key if extra_path else '' }"
print(path_models)
print(path)
print(output_models)

models/32features/mean/
data/32features/mean/
models_output/32features/mean/


In [46]:
# Read data
df_train = pd.read_csv(f"{path}train.csv", index_col=0)
df_valid = pd.read_csv(f"{path}valid.csv", index_col=0)
df_test = pd.read_csv(f"{path}test.csv", index_col=0)
print(len(df_train) + len(df_valid) + len(df_test))
print(len(df_train.columns))


train, valid, test = df_train.to_numpy(), df_valid.to_numpy(), df_test.to_numpy()

# y_**** contains the value of Survive7y as a list
# X_**** contains everything except for Survive7y as a list of list
X_train, y_train = train[:, :-1], train[:, -1]
X_valid, y_valid = valid[:, :-1], valid[:, -1]
X_test, y_test = test[:, :-1], test[:, -1]
feat_names = list(df_train.columns)
# Print how Survive7y are distribuited in each set
from collections import Counter
print(Counter(y_train))
print(Counter(y_valid))
print(Counter(y_test))

# All the numerical features that can be standardized
from utils import get_preprocess_std_num
preprocess_std = get_preprocess_std_num(feat_names)

# Preprocessed ready-to-use train and valid set
process_tmp = preprocess_std.fit(X_train)
X_train_std = process_tmp.transform(X_train)
X_valid_std = process_tmp.transform(X_valid)

#If you want to print the resulting df
# Note: You don't need to pass the _std to the train function. The function will call predict on the pipeline and transform the dataset accordingly to the transformer  
#df_scaled = pd.DataFrame(X_valid_std,columns = preprocess_std.get_feature_names_out())
#display(df_scaled)

6448
32
Counter({np.float64(1.0): 3494, np.float64(0.0): 505})
Counter({np.float64(1.0): 1082, np.float64(0.0): 143})
Counter({np.float64(1.0): 1082, np.float64(0.0): 142})


### Training


In [47]:
from functools import partial
from train import report, evaluate, train_and_evaluate
train_partial = partial(
    train_and_evaluate, 
    preprocess_std, 
    X_train=X_train,
    y_train=y_train,
    X_valid=X_valid,
    y_valid=y_valid,
    scoring="f1_macro", 
    iter=5000, 
    save=True,
    path_models = path_models,
    output_models = output_models
)

In [48]:
from sklearn.linear_model import LogisticRegression

hyperparams = hyperparameters["lr"] 
#Default is None (thus weight = 1). Balanced uses the formula n_samples / (n_classes * np.bincount(y))
model = LogisticRegression(class_weight="balanced")
train_partial(model=model, hyperparams=hyperparams, savename="lr")

/home/ileniag/ml4cad/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 63 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=63).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ileniag/ml4cad/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 63 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=63).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessi

Testing on training set:
              precision    recall  f1-score   support

         0.0      0.469     0.547     0.505       505
         1.0      0.933     0.911     0.922      3494

    accuracy                          0.865      3999
   macro avg      0.701     0.729     0.713      3999
weighted avg      0.874     0.865     0.869      3999

auc macro 0.854
confusion matrix
[[ 276  229]
 [ 312 3182]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.355     0.462     0.401       143
         1.0      0.926     0.889     0.907      1082

    accuracy                          0.839      1225
   macro avg      0.640     0.675     0.654      1225
weighted avg      0.859     0.839     0.848      1225

auc macro 0.809
confusion matrix
[[ 66  77]
 [120 962]]
Model rank: 1
Mean validation score: 0.707 (std: 0.014)
Parameters: {'model__C': 9, 'model__dual': True, 'model__max_iter': 475, 'model__penalty': 'l2', 'model__solver': 'libline

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('stand', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [49]:
from sklearn.svm import SVC
hyperparams = hyperparameters["svc"] 

model = SVC(class_weight="balanced", probability=True)
train_partial(model=model, hyperparams=hyperparams, savename="svc")

/home/ileniag/ml4cad/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1600).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/ileniag/ml4cad/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/ileniag/ml4cad/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=800).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/ileniag/ml4cad/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=800).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/ileniag/ml4cad/.venv/lib/python3.12/site-packages/sklearn/svm/_base.

Testing on training set:
              precision    recall  f1-score   support

         0.0      0.270     0.491     0.348       505
         1.0      0.917     0.808     0.859      3494

    accuracy                          0.768      3999
   macro avg      0.593     0.649     0.603      3999
weighted avg      0.835     0.768     0.794      3999

auc macro 0.279
confusion matrix
[[ 248  257]
 [ 672 2822]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.206     0.392     0.270       143
         1.0      0.909     0.800     0.851      1082

    accuracy                          0.753      1225
   macro avg      0.557     0.596     0.560      1225
weighted avg      0.827     0.753     0.783      1225

auc macro 0.342
confusion matrix
[[ 56  87]
 [216 866]]
Model rank: 1
Mean validation score: 0.674 (std: 0.002)
Parameters: {'model__C': 119, 'model__coef0': np.float64(0.6971844289451361), 'model__degree': 151, 'model__gamma': 'scale

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('stand', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [50]:
from sklearn.neighbors import KNeighborsClassifier

hyperparams = hyperparameters["knn"] 

model = KNeighborsClassifier()
train_partial(model=model, hyperparams=hyperparams, savename="knn")

Testing on training set:
              precision    recall  f1-score   support

         0.0      0.692     0.552     0.615       505
         1.0      0.937     0.965     0.951      3494

    accuracy                          0.912      3999
   macro avg      0.815     0.758     0.783      3999
weighted avg      0.906     0.912     0.908      3999

auc macro 0.941
confusion matrix
[[ 279  226]
 [ 124 3370]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.333     0.238     0.278       143
         1.0      0.903     0.937     0.920      1082

    accuracy                          0.856      1225
   macro avg      0.618     0.587     0.599      1225
weighted avg      0.836     0.856     0.845      1225

auc macro 0.663
confusion matrix
[[  34  109]
 [  68 1014]]
Model rank: 1
Mean validation score: 0.630 (std: 0.005)
Parameters: {'model__algorithm': 'ball_tree', 'model__leaf_size': 11, 'model__n_neighbors': 4, 'model__weights': 'unif

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('stand', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [51]:
from sklearn.ensemble import RandomForestClassifier

hyperparams = hyperparameters["rf"] 

model = RandomForestClassifier()
train_partial(model=model, hyperparams=hyperparams, savename="rf")

Testing on training set:
              precision    recall  f1-score   support

         0.0      0.871     0.966     0.916       505
         1.0      0.995     0.979     0.987      3494

    accuracy                          0.978      3999
   macro avg      0.933     0.973     0.952      3999
weighted avg      0.979     0.978     0.978      3999

auc macro 0.997
confusion matrix
[[ 488   17]
 [  72 3422]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.448     0.329     0.379       143
         1.0      0.914     0.946     0.930      1082

    accuracy                          0.874      1225
   macro avg      0.681     0.638     0.655      1225
weighted avg      0.860     0.874     0.866      1225

auc macro 0.810
confusion matrix
[[  47   96]
 [  58 1024]]
Model rank: 1
Mean validation score: 0.711 (std: 0.001)
Parameters: {'model__class_weight': 'balanced', 'model__criterion': 'entropy', 'model__max_features': 'sqrt', 'model__

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('stand', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [52]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

hyperparams = hyperparameters["adaboost"] 

model = AdaBoostClassifier()
train_partial(model=model, hyperparams=hyperparams, savename="adaboost")

Testing on training set:
              precision    recall  f1-score   support

         0.0      0.653     0.283     0.395       505
         1.0      0.904     0.978     0.940      3494

    accuracy                          0.890      3999
   macro avg      0.779     0.631     0.667      3999
weighted avg      0.873     0.890     0.871      3999

auc macro 0.872
confusion matrix
[[ 143  362]
 [  76 3418]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.493     0.245     0.327       143
         1.0      0.906     0.967     0.936      1082

    accuracy                          0.882      1225
   macro avg      0.700     0.606     0.631      1225
weighted avg      0.858     0.882     0.865      1225

auc macro 0.802
confusion matrix
[[  35  108]
 [  36 1046]]
Model rank: 1
Mean validation score: 0.677 (std: 0.005)
Parameters: {'model__learning_rate': np.float64(1.1674688399832753), 'model__n_estimators': 58}

Model rank: 2
Mean va

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('stand', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [53]:
from sklearn.neural_network import MLPClassifier
import random

hyperparams = hyperparameters["nn"] 

model = MLPClassifier()
train_partial(model=model, hyperparams=hyperparams, savename="nn")

Testing on training set:
              precision    recall  f1-score   support

         0.0      0.598     0.386     0.469       505
         1.0      0.916     0.963     0.938      3494

    accuracy                          0.890      3999
   macro avg      0.757     0.674     0.704      3999
weighted avg      0.876     0.890     0.879      3999

auc macro 0.856
confusion matrix
[[ 195  310]
 [ 131 3363]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.533     0.343     0.417       143
         1.0      0.917     0.960     0.938      1082

    accuracy                          0.888      1225
   macro avg      0.725     0.651     0.678      1225
weighted avg      0.872     0.888     0.877      1225

auc macro 0.814
confusion matrix
[[  49   94]
 [  43 1039]]
Model rank: 1
Mean validation score: 0.706 (std: 0.002)
Parameters: {'model__alpha': np.float64(0.449288214583039), 'model__early_stopping': True, 'model__hidden_layer_sizes'

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('stand', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [54]:
from sklearn.ensemble import GradientBoostingClassifier

hyperparams = hyperparameters["gb"] 

model = GradientBoostingClassifier()
train_partial(model=model, hyperparams=hyperparams, savename="gb")

Testing on training set:
              precision    recall  f1-score   support

         0.0      0.741     0.386     0.508       505
         1.0      0.917     0.981     0.948      3494

    accuracy                          0.905      3999
   macro avg      0.829     0.683     0.728      3999
weighted avg      0.895     0.905     0.892      3999

auc macro 0.899
confusion matrix
[[ 195  310]
 [  68 3426]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.582     0.273     0.371       143
         1.0      0.910     0.974     0.941      1082

    accuracy                          0.892      1225
   macro avg      0.746     0.623     0.656      1225
weighted avg      0.872     0.892     0.875      1225

auc macro 0.816
confusion matrix
[[  39  104]
 [  28 1054]]
Model rank: 1
Mean validation score: 0.693 (std: 0.006)
Parameters: {'model__learning_rate': np.float64(0.06384160006481765), 'model__max_depth': 3, 'model__max_features': No

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('stand', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [55]:
#Don't run this in jupyter within vscode, run this with notebooks within browsers.
import os
#os.environ['KMP_DUPLICATE_LIB_OK']='True'

import xgboost as xgb

hyperparams = hyperparameters["xgb"] 

model = xgb.XGBClassifier(n_jobs=1)
train_partial(model=model, hyperparams=hyperparams, savename="xgb")

/home/ileniag/ml4cad/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:05:33] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "gamma", "max_depth", "subsample" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/ileniag/ml4cad/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:05:33] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "gamma", "max_depth", "subsample" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/ileniag/ml4cad/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:05:33] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "gamma", "max_depth", "subsample" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/ileniag/ml4cad/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:05:33] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "gamma", "max_depth", "subsample" } are not used.

  bst.update(dtrain, ite

Testing on training set:
              precision    recall  f1-score   support

         0.0      0.549     0.552     0.551       505
         1.0      0.935     0.934     0.935      3494

    accuracy                          0.886      3999
   macro avg      0.742     0.743     0.743      3999
weighted avg      0.887     0.886     0.886      3999

auc macro 0.879
confusion matrix
[[ 279  226]
 [ 229 3265]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.392     0.434     0.412       143
         1.0      0.924     0.911     0.918      1082

    accuracy                          0.856      1225
   macro avg      0.658     0.672     0.665      1225
weighted avg      0.862     0.856     0.859      1225

auc macro 0.817
confusion matrix
[[ 62  81]
 [ 96 986]]
Model rank: 1
Mean validation score: 0.718 (std: 0.001)
Parameters: {'model__alpha': np.float64(0.07009634744640048), 'model__booster': 'dart', 'model__eta': np.float64(0.2632802

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('stand', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [56]:

mean_path = f"data/27features/"
df_train = pd.read_csv(f"{mean_path}train.csv")
df_valid = pd.read_csv(f"{mean_path}valid.csv")
df_test = pd.read_csv(f"{mean_path}test.csv")
sum_valid = 0
sum_test = 0
print(1 in df_valid.iloc[:,0].to_numpy())
for val in df_train.iloc[:,0].to_numpy():
    if val in df_valid.iloc[:,0].to_numpy():
        print(val)
        sum_valid +=1
    if val in df_test.iloc[:,0].to_numpy():
        sum_test +=1
print("#######################")
print(sum_valid)
print(sum_test)

False
#######################
0
0
